In [ ]:
from train import train
env_name = 'mountain_car'
train('mountain_car', num_examples=640, mode=1, num_examples_phase2=1, 
              retrain_phase1=True, retrain_phase2=True)

In [ ]:
import utils
import torch
import numpy as np
from model_nets import HDNet
from torchdiffeq import odeint_adjoint as odeint


env_name = "mountain_car"
num_trajs = 1
time_steps = list(np.arange(0, 1.0, 0.05))



_, adj_net, hnet, hnet_decoder, _, _ = utils.get_architectures(
    arch_file="models/architectures.csv", env_name=env_name
)
env = utils.get_environment(env_name)
adj_net.load_state_dict(torch.load("models/" + env_name + "/adjoint.pth"))
hnet.load_state_dict(torch.load("models/" + env_name + "/hamiltonian_dynamics.pth"))
HDnet = HDNet(hnet=hnet)

q = torch.tensor(env.sample_q(num_trajs), dtype=torch.float)
p = adj_net(q)
qp = torch.cat((q, p), axis=1)
traj = odeint(HDnet, qp, torch.tensor(time_steps, requires_grad=False))

In [16]:
traj.shape

torch.Size([20, 1, 4])

In [13]:
traj[:5, 0, :2]

tensor([[-0.4494, -0.0113],
        [-0.4356,  0.0160],
        [-0.4220,  0.0435],
        [-0.4086,  0.0710],
        [-0.3955,  0.0986]], grad_fn=<SliceBackward0>)

In [9]:
from torch.autograd.functional import jacobian

params = tuple(hnet.parameters()) + tuple(adj_net.parameters())

def rollout(*param_list):
    with torch.no_grad():
        for p, new_p in zip(params, param_list):
            p.copy_(new_p)

    trajectory = odeint(HDNet, qp, torch.tensor(time_steps))
    return trajectory.reshape(-1)

J = jacobian(rollout, params, create_graph=False)
env.W_ksi(traj[:,0,:2], 0.1).shape

ValueError: func must be an instance of nn.Module to specify the adjoint parameters; alternatively they can be specified explicitly via the `adjoint_params` argument. If there are no parameters then it is allowable to set `adjoint_params=()`.

In [4]:
traj[:,0,:2].shape

torch.Size([20, 2])